# Simple Tests

Some simple tests can be performed to test the performance and the behavior of the `EUNN` and the `EURNN` recurrent unit.

## Imports

In [1]:
import torch
import numpy as np
import sys; sys.path.append('..')
from torch_eunn import EUNN

np.set_printoptions(precision=2, suppress=True)

## Speed

We can compare the execution speed of the `EUNN`, which should act like a unitary matrix to a normal `torch.Linear` layer.

In [2]:
class SpeedTestModule(torch.nn.Module):
    def __init__(self, type, num_hidden, capacity=None):
        super(SpeedTestModule, self).__init__()
        self.type = type
        if self.type == 'EUNN':
            self.layer = EUNN(num_hidden, capacity=num_hidden if capacity is None else capacity)
        elif self.type == 'LINEAR':
            self.layer = torch.nn.Linear(2*num_hidden, 2*num_hidden)
        else:
            raise ValueError
        self.lossfunc = torch.nn.MSELoss()
    def forward(self, x):
        ''' perform forward on a complex valued tensor (complex index = last) '''
        if self.type == 'EUNN':
            return self.layer(x)
        else: # linear only works on a real valued tensor, therefore, stack the last two indices
            return self.layer(x.view(x.shape[0], 2*num_hidden)).view(x.shape[0], num_hidden, 2)
    def test(self, x):
        ''' perform a backward and a forward pass for speed comparison '''
        y = self(x)
        loss = ((1-y)**2).mean()
        loss.backward()

In [3]:
batch_size = 5
num_hidden = 500
x = torch.randn(batch_size, num_hidden, 2)
test_eunn_fc = SpeedTestModule('EUNN', num_hidden) # full capacity eunn
test_eunn_c2 = SpeedTestModule('EUNN', num_hidden, 2) # capacity 2 eunn
test_lin = SpeedTestModule('LINEAR', num_hidden)

# test
%time test_lin.test(x)
%time test_eunn_c2.test(x)
%time test_eunn_fc.test(x)

CPU times: user 77.7 ms, sys: 22.6 ms, total: 100 ms
Wall time: 26.6 ms
CPU times: user 21.4 ms, sys: 0 ns, total: 21.4 ms
Wall time: 3.02 ms
CPU times: user 1.48 s, sys: 126 ms, total: 1.6 s
Wall time: 452 ms


## Unitarity

The action of a EUNNLayer should always be unitary.

In [4]:
# dimensionality of the cell
num_hidden = 50

# create new cell
cell = EUNN(num_hidden)

# get result of action of cell on identity matrix:
x = torch.stack([torch.eye(num_hidden, num_hidden), torch.zeros(num_hidden, num_hidden)], -1)
y = cell(x)
y = y[...,0].detach().numpy() + 1j*y[...,1].detach().numpy()

# check unitarity of result
print(np.abs(y@y.T.conj()))

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


We see that the operation of a `EUNN` is clearly unitary.

## Universality

Next we check if a full capacity cell can approximate any unitary matrix

In [5]:
# dimensionality of the cell
num_hidden = 10

# create new cell
cell = EUNN(num_hidden, num_hidden)

# create unitary matrix to approximate
U, _, _ = np.linalg.svd(np.random.randn(num_hidden,num_hidden) + 1j*np.random.randn(num_hidden,num_hidden))
U_torch = torch.stack([
    torch.tensor(np.real(U.T.conj()), dtype=torch.float32),
    torch.tensor(np.imag(U.T.conj()), dtype=torch.float32),
], -1)

# create the target
# the cell needs to be trained such that action of the cell on U.T.conj() yields the identity
I_torch = torch.stack([
    torch.eye(num_hidden),   
    torch.zeros((num_hidden,num_hidden)),
], -1)

# criterion & optimizer
lossfunc = torch.nn.MSELoss()
optimizer = torch.optim.Adam(cell.parameters(), lr=0.0020)

# training
for _ in range(10000):
    optimizer.zero_grad()
    I_approx = cell(U_torch)
    loss = lossfunc(I_approx, I_torch)
    loss.backward()
    optimizer.step()

result = I_approx[...,0].detach().numpy() + 1j*I_approx[...,1].detach().numpy()

print(abs(result)**2)

[[0.95 0.   0.   0.   0.   0.   0.01 0.01 0.01 0.  ]
 [0.   0.97 0.   0.   0.   0.   0.   0.01 0.01 0.  ]
 [0.   0.   0.97 0.   0.   0.   0.   0.02 0.01 0.  ]
 [0.   0.   0.   0.99 0.   0.   0.   0.01 0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.99 0.   0.   0.   0.  ]
 [0.01 0.   0.   0.   0.   0.   0.98 0.   0.   0.  ]
 [0.01 0.01 0.02 0.   0.   0.   0.   0.96 0.   0.  ]
 [0.01 0.01 0.01 0.   0.   0.   0.   0.   0.96 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]


We see that we can approximate the matrix U.